In [1]:
!pip install --upgrade pip

  Using cached pip-22.1.2-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
D:\Anaconda\python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow
!pip install opencv-python    
!pip install pillow
!pip install scikit-image

In [3]:
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(100,100))
    images=[row,scale_out,scale_in,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,100,100,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,96],[50,50])

[1493, 2780, 1583]
3747
14988
Epoch 1/100
100/100 [==============================] - 126s 1s/step - loss: 1.0394 - accuracy: 0.4949 - val_loss: 0.7074 - val_accuracy: 0.6841
Epoch 2/100
100/100 [==============================] - 119s 1s/step - loss: 0.9541 - accuracy: 0.5370 - val_loss: 0.6437 - val_accuracy: 0.7108
Epoch 3/100
100/100 [==============================] - 119s 1s/step - loss: 0.8794 - accuracy: 0.5850 - val_loss: 0.5700 - val_accuracy: 0.7620
Epoch 4/100
100/100 [==============================] - 121s 1s/step - loss: 0.6388 - accuracy: 0.7341 - val_loss: 0.5650 - val_accuracy: 0.7663
Epoch 5/100
100/100 [==============================] - 111s 1s/step - loss: 0.5989 - accuracy: 0.7485 - val_loss: 0.5728 - val_accuracy: 0.7684
Epoch 6/100
100/100 [==============================] - 106s 1s/step - loss: 0.5744 - accuracy: 0.7591 - val_loss: 0.5677 - val_accuracy: 0.7620
Epoch 7/100
100/100 [==============================] - 106s 1s/step - loss: 0.5535 - accuracy: 0.7686 - va

In [1]:
#200,200
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(200,200))
    images=[row,scale_out,scale_in,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim200x200'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,200,200,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,96,128],[50,50])

[1493, 2780, 1583]
3747
14988
Epoch 1/100
100/100 [==============================] - 745s 7s/step - loss: 1.0712 - accuracy: 0.4722 - val_loss: 0.9943 - val_accuracy: 0.4749
Epoch 2/100
100/100 [==============================] - 741s 7s/step - loss: 0.9984 - accuracy: 0.5189 - val_loss: 0.6196 - val_accuracy: 0.7225
Epoch 3/100
100/100 [==============================] - 771s 8s/step - loss: 0.9554 - accuracy: 0.5417 - val_loss: 0.5324 - val_accuracy: 0.7705
Epoch 4/100
100/100 [==============================] - 773s 8s/step - loss: 0.9418 - accuracy: 0.5483 - val_loss: 0.4875 - val_accuracy: 0.7919
Epoch 5/100
100/100 [==============================] - 787s 8s/step - loss: 0.9313 - accuracy: 0.5500 - val_loss: 0.4759 - val_accuracy: 0.7940
Epoch 6/100
100/100 [==============================] - 792s 8s/step - loss: 0.9305 - accuracy: 0.5520 - val_loss: 0.4526 - val_accuracy: 0.8036
Epoch 7/100
100/100 [==============================] - 812s 8s/step - loss: 0.9229 - accuracy: 0.5532 - va

#### Now we wil test it with 2 dense layers smaller batches

In [8]:
#200,200
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(200,200))
    images=[row,scale_out,scale_in,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)


from sklearn.utils import shuffle
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim200x200'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        keeped=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        keeped_Y=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        if(i==0):
            keeped=keeped[0:int(len(keeped)/2)]
            keeped_Y=keeped_Y[0:int(len(keeped_Y)/2)]                 
        TVT[i]+=keeped
        Y_TVT[i]+=keeped_Y
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented
TVT[0],Y_TVT[0]=shuffle(TVT[0],Y_TVT[0])

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,200,200,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            #hidden=layers.BatchNormalization()(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,100,[pow(2,5+i) for i in range(4)],[40,30])

[1493, 2780, 1583]
1872
7488
Epoch 1/100
75/75 [==============================] - 459s 6s/step - loss: 1.0742 - accuracy: 0.4675 - val_loss: 1.0775 - val_accuracy: 0.4749
Epoch 2/100
75/75 [==============================] - 444s 6s/step - loss: 1.0665 - accuracy: 0.4748 - val_loss: 1.0695 - val_accuracy: 0.4749
Epoch 3/100
75/75 [==============================] - 440s 6s/step - loss: 1.0538 - accuracy: 0.4752 - val_loss: 0.9448 - val_accuracy: 0.4899
Epoch 4/100
75/75 [==============================] - 440s 6s/step - loss: 1.0173 - accuracy: 0.5051 - val_loss: 0.7240 - val_accuracy: 0.6884
Epoch 5/100
75/75 [==============================] - 437s 6s/step - loss: 0.9824 - accuracy: 0.5294 - val_loss: 0.6458 - val_accuracy: 0.7364
Epoch 6/100
75/75 [==============================] - 441s 6s/step - loss: 0.9612 - accuracy: 0.5385 - val_loss: 0.6413 - val_accuracy: 0.7417
Epoch 7/100
75/75 [==============================] - 438s 6s/step - loss: 0.9544 - accuracy: 0.5387 - val_loss: 0.5628 

#### Let's  set batch size 75

In [9]:
#200,200
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(200,200))
    images=[row,scale_out,scale_in,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)


from sklearn.utils import shuffle
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim200x200'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        keeped=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        keeped_Y=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        if(i==0):
            keeped=keeped[0:int(len(keeped)/2)]
            keeped_Y=keeped_Y[0:int(len(keeped_Y)/2)]                 
        TVT[i]+=keeped
        Y_TVT[i]+=keeped_Y
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented
TVT[0],Y_TVT[0]=shuffle(TVT[0],Y_TVT[0])

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,200,200,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            #hidden=layers.BatchNormalization()(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,75,[pow(2,5+i) for i in range(4)],[40,30])

[1493, 2780, 1583]
1872
7488
Epoch 1/100
100/100 [==============================] - 421s 4s/step - loss: 1.0729 - accuracy: 0.4697 - val_loss: 1.0475 - val_accuracy: 0.4749
Epoch 2/100
100/100 [==============================] - 395s 4s/step - loss: 1.0473 - accuracy: 0.4892 - val_loss: 0.7916 - val_accuracy: 0.6446
Epoch 3/100
100/100 [==============================] - 405s 4s/step - loss: 0.9947 - accuracy: 0.5236 - val_loss: 0.6895 - val_accuracy: 0.7279
Epoch 4/100
100/100 [==============================] - 374s 4s/step - loss: 0.9758 - accuracy: 0.5339 - val_loss: 0.6033 - val_accuracy: 0.7460
Epoch 5/100
100/100 [==============================] - 373s 4s/step - loss: 0.9538 - accuracy: 0.5451 - val_loss: 0.6105 - val_accuracy: 0.7449
Epoch 6/100
100/100 [==============================] - 367s 4s/step - loss: 0.9450 - accuracy: 0.5481 - val_loss: 0.6647 - val_accuracy: 0.7481
Epoch 7/100
100/100 [==============================] - 375s 4s/step - loss: 0.9295 - accuracy: 0.5554 - val

#### We dropped data.That was wrong.Next step we will try to limit the augmentations

In [13]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

553476096/553467096 [==============================] - 90s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

#### Try with ImageDataGenerator

In [17]:
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img,img_to_array
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

datagen = ImageDataGenerator(
        rotation_range=[10,20],
        zoom_range=[-0.2,0.2],
        fill_mode="reflect",
        horizontal_flip=True
    )

def process(X,file,gen):
    img=load_img(
        file,color_mode="grayscale"
    )
    X.append(img)
    
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)
    it=gen.flow(img_array, batch_size=3)
    for i in range(2):
        batch=it.next()
        X.append(load_img(batch[0].astype("uint8")),color_mode="grayscale")
    
    

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    for i in range(3):
        Y[k].append(str(k))
    process(classes[k],file,datagen)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

#train_datagen.fit(Train)
#validation_datagen.fit(Valid)
#test_datagen.fit(Test)

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            i2=i+1
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(validation_datagen.flow(TVT[1]),Y_TVT[1]),epochs=epochs_num,
              batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(test_datagen.flow(TVT[2]), Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,96],[50,50])



   
        


TypeError: bad operand type for unary -: 'list'